# CIMD in OAuth — Minimal Flow Simulation

This notebook extends the minimal OAuth simulation
by introducing Client-ID Metadata Documents (CIMD).

Goal:
- Replace static client_id with URL-based identity
- Enforce CIMD invariants mechanically
- Keep the OAuth flow unchanged


## What CIMD Changes (and What It Does Not)

Changes:
- client_id is now a URL
- Authorization Server must fetch metadata
- Client identity becomes dynamic

Unchanged:
- Authorization Code flow
- PKCE
- Redirect URI matching
- Token issuance


In [22]:
# Simulated remote CIMD metadata documents
REMOTE_METADATA = {
    "https://agent.example.com/identity.json": {
        "client_id": "https://agent.example.com/identity.json",
        "redirect_uris": ["https://agent.example.com/callback"],
        "grant_types": ["authorization_code"],
        "scope": "research.read"
    }
}

AUTHORIZATION_CODES = {}
ISSUED_TOKENS = {}

In [9]:
print(REMOTE_METADATA.values())
print(REMOTE_METADATA.get("https://agent.example.com/identity.json"))

dict_values([{'client_id': 'https://agent.example.com/identity.json', 'redirect_uris': ['https://agent.example.com/callback'], 'grant_types': ['authorization_code'], 'scope': 'research.read'}])
{'client_id': 'https://agent.example.com/identity.json', 'redirect_uris': ['https://agent.example.com/callback'], 'grant_types': ['authorization_code'], 'scope': 'research.read'}


In [15]:
def validate_cimd_url(client_id: str):
    assert client_id.startswith("https://"), "client_id must be HTTPS"


def fetch_client_metadata(client_id: str):
    validate_cimd_url(client_id)

    metadata = REMOTE_METADATA.get(client_id)
    assert metadata, "metadata not found"

    # Invariant 3: metadata must self-identify
    assert metadata["client_id"] == client_id, "client_id mismatch"

    return metadata


In [ ]:
# Updated authorization request (Code)
import secrets

def authorization_request_cimd(
    client_id: str,
    redirect_uri: str,
    scope: str,
    code_challenge: str
):
    metadata = fetch_client_metadata(client_id)
    
    # Invariant 4: redirect uri must match metadata
    assert redirect_uri in metadata["redirect_uris"], "redirect_uri mismatch"
    
    assert code_challenge, "PKCE required"
    
    code = secrets.token_urlsafe(16)
    # print("(secrets.tokenUrlsafe)code: ", code)
    
    AUTHORIZATION_CODES[code] = {
        "client_id": client_id,
        "redirect_uri": redirect_uri,
        "scope": scope,
        "code_challenge": code_challenge
    }
    
    return code

In [13]:
# this cell stays the same
import hashlib
import base64

def s256(verifier: str)-> str:
    digest = hashlib.sha256(verifier.encode()).digest()
    return base64.urlsafe_b64encode(digest).rstrip(b'=').decode('ascii')

def token_request(
    code: str,
    client_id: str,
    redirect_uri: str,
    code_verifier: str
):
    record = AUTHORIZATION_CODES.pop(code, None)
    assert record, "Invalid authorization code"
    
    assert record["client_id"] == client_id
    assert record["redirect_uri"] == redirect_uri
    
    assert s256(code_verifier) == record["code_challenge"], "PKCE failed"
    
    token = secrets.token_urlsafe(24)
    ISSUED_TOKENS[token] = {
        "client_id": client_id,
        "scope": record["scope"]
    }
    
    return token

In [16]:
# Happy path simulation

CLIENT_ID = "https://agent.example.com/identity.json"
REDIRECT_URI = "https://agent.example.com/callback"
SCOPE = "research.read"

code_verifier = "random-secret"
code_challenge = s256(code_verifier)

auth_code = authorization_request_cimd(
    client_id=CLIENT_ID,
    redirect_uri=REDIRECT_URI,
    scope=SCOPE,
    code_challenge=code_challenge
)

access_token = token_request(
    auth_code,
    client_id=CLIENT_ID,
    redirect_uri=REDIRECT_URI,
    code_verifier=code_verifier
)

access_token

'LhrPZhnQS_7u-X0eOAkXREaGX9QuGUpf'

In [ ]:
# wrong id 

authorization_request_cimd(
    "https://evil.example.com/identity.json",
    REDIRECT_URI,
    SCOPE,
    code_challenge
)


AssertionError: metadata not found

In [ ]:
# Meta data impersonation

# print(REMOTE_METADATA["https://agent.example.com/identity.json"]["client_id"])
REMOTE_METADATA["https://agent.example.com/identity.json"]["client_id"] = "https://other-agent.example.com/identity.json"
# print(REMOTE_METADATA["https://agent.example.com/identity.json"]["client_id"])

authorization_request_cimd(
    CLIENT_ID,
    REDIRECT_URI,
    SCOPE,
    code_challenge
)

https://agent.example.com/identity.json
https://other-agent.example.com/identity.json


AssertionError: client_id mismatch